### Solution

In [1]:
class BubbleSortCyclicSolver:
    
    def __init__(self):
        self.perm = []
        self.n = 0
        self.solution = []
        self.pos = 0
        self.left_border = 0
        self.right_border = 0

    def move_left(self):
        """Moves left, updating the solution list accordingly."""
        if self.solution[-1] != 'L':
            self.solution.append('R')
        else:
            self.solution.pop() # this works only for n=2 and n=3
        self.pos = (self.pos - 1 + self.n) % self.n

    def move_right(self):
        """Moves right, updating the solution list accordingly."""
        if self.solution[-1] != 'R':
            self.solution.append('L')
        else:
            self.solution.pop() # this works only for n=2 and n=3
        self.pos = (self.pos + 1) % self.n

    def swap(self):
        """Swaps the current position."""
        self.solution.append('X')

    def solve(self, perm):
        """Executes the sorting process and returns the sequence of actions."""
        self.perm = perm
        self.n = len(perm)
        self.solution = ['X', 'L']
        self.pos = 1
        self.left_border = 2
        self.right_border = self.n - 1

        if self.n % 4 != 2 or self.n == 2:
            self._solve_case_one()
        else:
            self._solve_case_two()
        return self.solution

    def _solve_case_one(self):
        while self.right_border - self.left_border > self.n // 2:
            while self.pos != self.right_border:
                self.swap()
                self.move_left()
            self.right_border -= 1

            while self.pos != self.left_border:
                self.swap()
                self.move_right()
            self.left_border += 1

        while self.right_border - self.left_border > 0:
            while self.pos != self.right_border - 1:
                self.move_right()
                self.swap()
            self.right_border -= 1

            while self.pos != self.left_border:
                self.move_left()
                self.swap()
            self.left_border += 1

        while self.pos != 0:
            self.move_left()

    def _solve_case_two(self):
        while self.right_border - self.left_border > self.n // 2:
            while self.pos != self.right_border:
                self.swap()
                self.move_left()
            self.right_border -= 1

            while self.pos != self.left_border:
                self.swap()
                self.move_right()
            self.left_border += 1

        while self.pos != self.right_border:
            self.swap()
            self.move_left()
        self.left_border += 1

        while self.right_border - self.left_border > 0:
            while self.pos != self.left_border:
                self.move_left()
                self.swap()
            self.left_border += 1

            while self.pos != self.right_border - 1:
                self.move_right()
                self.swap()
            self.right_border -= 1

        while self.pos != 0:
            self.move_left()
        self.move_right()

### Testers

In [2]:
def L(a):
    result = a[1:]
    result.append(a[0])
    return result

def R(a):
    result = [a[-1]]
    result.extend(a[:-1])
    return result

def X(a):
    result = [a[1], a[0]]
    result.extend(a[2:])
    return result

ops = {
    'L': L,
    'R': R,
    'X': X,
}

def apply_solution(perm, solution):
    result = perm
    for op in solution:
        result = ops[op](result)

    return result

def test(n, solver):
    score = 0
    for i in range(2, n + 1):
        sorted_perm = [x for x in range(i)]
        perm = R(R(sorted_perm[::-1]))
        moves = solver.solve(perm)
        result = apply_solution(perm, moves)

        score += len(moves)
        if result != sorted_perm:
            print('WA at n={}, got {}'.format(i, result))
        # if len(moves) != i * (i - 1) / 2:
        #     print(i, len(moves), len(moves) - i * (i - 1) / 2)

    print('Total score: ', score)

def print_permutation(perm, pos):
    
    # Determine the width based on the maximum number in the array
    width = len(str(max(perm))) + 1

    perm_shifted = perm[-pos:] + perm[:-pos]
    # Build the formatted list
    formatted = []
    for i, num in enumerate(perm_shifted):
        if i == pos:
            formatted.append(f"|{str(num)}|".rjust(width + 1))
        elif i == pos + 1:
            formatted.append(str(num).rjust(width - 1))
        else:
            formatted.append(str(num).rjust(width))

    print("".join(formatted))

def visualize_solution(perm, solution, start_pos=0, tab_len=1):
    result = perm
    pos = start_pos
    n = len(perm)

    print(' ' * 3, end='')
    print_permutation(result, pos)
    for op in solution:
        result = ops[op](result)
        if op == 'R':
            pos = (pos - 1 + n) % n
        elif op == 'L':
            pos = (pos + 1) % n

        print(op + ': ', end='')
        print_permutation(result, pos)

    print()

def visualize_solution_at_n(n, solver, start_pos=0, tab_len=1):
    sorted_perm = [x for x in range(n)]
    perm = R(R(sorted_perm[::-1]))
    moves = solver.solve(perm)
    result = apply_solution(perm, moves)

    print('Solution = {}'.format(''.join(moves)))
    print('n={}, len(moves)={}, diff with optimal={}'.format(
        n, 
        len(moves), 
        len(moves) - n * (n - 1) / 2)
    )

    visualize_solution(perm, moves, start_pos, tab_len)

In [3]:
test(200, BubbleSortCyclicSolver())

Total score:  1333298


In [4]:
import pandas as  pd

test_df = pd.read_csv('/kaggle/input/lrx-oeis-a-186783-brainstorm-math-conjecture/test.csv')

score = 0
solution = []
for idx in range(len(test_df)):
    n = test_df.n.values[idx]
    perm = eval('[' + test_df.permutation.values[idx] + ']')

    moves = BubbleSortCyclicSolver().solve(perm)
    solution.append((n, ''.join(moves)))
    score += len(moves)

print('Total score: ', score)
solution = pd.DataFrame(solution)
solution.columns = ['n','solution']
solution.to_csv('submission.csv', index=False)

Total score:  1333298


### Explanation

#### Naive

This algorithm is an optimized version of [Bubble sort](https://en.wikipedia.org/wiki/Bubble_sort). So in naive implenetation it will look like this:

In [5]:
class BubbleSortSolver(BubbleSortCyclicSolver):
    """Implements a basic bubble sort movement strategy."""
    
    def solve(self, perm):
        self.perm = perm
        self.n = len(perm)
        self.solution = ['L', 'L']
        self.pos = 0

        for i in range(self.n):
            for _ in range(self.n - 1 - i):
                self.swap()
                self.move_right()
            
            while self.pos != 0:
                self.move_left()
        
        return self.solution

In [6]:
test(200, BubbleSortSolver())

Total score:  3960498


WLOG, we will consider a permutation as an array of numbers. We will consider shift operations not as shifting the array, but as shifting the cursor-pointer on the array (see visualization)

In [7]:
visualize_solution_at_n(7, BubbleSortSolver(), start_pos = -2)

Solution = LLXLXLXLXLXLXRRRRRXLXLXLXLXRRRRXLXLXLXRRRXLXLXRRXLXRX
n=7, len(moves)=53, diff with optimal=32.0
    6 5 4 3 2 1 0
L:  6 5 4 3 2 1|0|
L: |6|5 4 3 2 1 0
X: |5|6 4 3 2 1 0
L:  5|6|4 3 2 1 0
X:  5|4|6 3 2 1 0
L:  5 4|6|3 2 1 0
X:  5 4|3|6 2 1 0
L:  5 4 3|6|2 1 0
X:  5 4 3|2|6 1 0
L:  5 4 3 2|6|1 0
X:  5 4 3 2|1|6 0
L:  5 4 3 2 1|6|0
X:  5 4 3 2 1|0|6
R:  5 4 3 2|1|0 6
R:  5 4 3|2|1 0 6
R:  5 4|3|2 1 0 6
R:  5|4|3 2 1 0 6
R: |5|4 3 2 1 0 6
X: |4|5 3 2 1 0 6
L:  4|5|3 2 1 0 6
X:  4|3|5 2 1 0 6
L:  4 3|5|2 1 0 6
X:  4 3|2|5 1 0 6
L:  4 3 2|5|1 0 6
X:  4 3 2|1|5 0 6
L:  4 3 2 1|5|0 6
X:  4 3 2 1|0|5 6
R:  4 3 2|1|0 5 6
R:  4 3|2|1 0 5 6
R:  4|3|2 1 0 5 6
R: |4|3 2 1 0 5 6
X: |3|4 2 1 0 5 6
L:  3|4|2 1 0 5 6
X:  3|2|4 1 0 5 6
L:  3 2|4|1 0 5 6
X:  3 2|1|4 0 5 6
L:  3 2 1|4|0 5 6
X:  3 2 1|0|4 5 6
R:  3 2|1|0 4 5 6
R:  3|2|1 0 4 5 6
R: |3|2 1 0 4 5 6
X: |2|3 1 0 4 5 6
L:  2|3|1 0 4 5 6
X:  2|1|3 0 4 5 6
L:  2 1|3|0 4 5 6
X:  2 1|0|3 4 5 6
R:  2|1|0 3 4 5 6
R: |2|1 0 3 4 5 6
X: |1|2 0

#### Bidirectional

At least it works. First optimization - make bidirectional. We just move back to pos=0 now. Let's move some element with us

In [8]:
class BubbleSortBidirectionalSolver(BubbleSortCyclicSolver):
    """Implements a basic bubble sort movement strategy."""
    
    def solve(self, perm):
        self.perm = perm
        self.n = len(perm)
        self.solution = ['L', 'L']
        self.pos = 0

        for i in range(self.n // 2 * 2):
            if i % 2 == 0:
                for _ in range(self.n - 1 - i):
                    self.swap()
                    self.move_right()
                self.move_left()
                self.move_left()
            else:
                for _ in range(self.n - 1 - i):
                    self.swap()
                    self.move_left()
                self.move_right()
                self.move_right()

        while self.pos != 0:
            self.move_left()
        
        return self.solution

In [9]:
test(200, BubbleSortBidirectionalSolver())

Total score:  2676600


In [10]:
visualize_solution_at_n(7, BubbleSortBidirectionalSolver(), start_pos = -2)

Solution = LLXLXLXLXLXLXRXRXRXRXRXLXLXLXLXRXRXRXLXLXRXRR
n=7, len(moves)=45, diff with optimal=24.0
    6 5 4 3 2 1 0
L:  6 5 4 3 2 1|0|
L: |6|5 4 3 2 1 0
X: |5|6 4 3 2 1 0
L:  5|6|4 3 2 1 0
X:  5|4|6 3 2 1 0
L:  5 4|6|3 2 1 0
X:  5 4|3|6 2 1 0
L:  5 4 3|6|2 1 0
X:  5 4 3|2|6 1 0
L:  5 4 3 2|6|1 0
X:  5 4 3 2|1|6 0
L:  5 4 3 2 1|6|0
X:  5 4 3 2 1|0|6
R:  5 4 3 2|1|0 6
X:  5 4 3 2|0|1 6
R:  5 4 3|2|0 1 6
X:  5 4 3|0|2 1 6
R:  5 4|3|0 2 1 6
X:  5 4|0|3 2 1 6
R:  5|4|0 3 2 1 6
X:  5|0|4 3 2 1 6
R: |5|0 4 3 2 1 6
X: |0|5 4 3 2 1 6
L:  0|5|4 3 2 1 6
X:  0|4|5 3 2 1 6
L:  0 4|5|3 2 1 6
X:  0 4|3|5 2 1 6
L:  0 4 3|5|2 1 6
X:  0 4 3|2|5 1 6
L:  0 4 3 2|5|1 6
X:  0 4 3 2|1|5 6
R:  0 4 3|2|1 5 6
X:  0 4 3|1|2 5 6
R:  0 4|3|1 2 5 6
X:  0 4|1|3 2 5 6
R:  0|4|1 3 2 5 6
X:  0|1|4 3 2 5 6
L:  0 1|4|3 2 5 6
X:  0 1|3|4 2 5 6
L:  0 1 3|4|2 5 6
X:  0 1 3|2|4 5 6
R:  0 1|3|2 4 5 6
X:  0 1|2|3 4 5 6
R:  0|1|2 3 4 5 6
R: |0|1 2 3 4 5 6



It's better, but we don't use the fact that our array is cyclic. Let's do this

#### Cyclic

Here we do the same as before, but we go the closer way to the target position (using cyclic properties). Check visualization for better understanding

Code is the same as in the first code-block
There is also check for n=4k+2 case, just because I'm too lazy to do it the proper way

In [11]:
class BubbleSortCyclicSolver:
    
    def __init__(self):
        self.perm = []
        self.n = 0
        self.solution = []
        self.pos = 0
        self.left_border = 0
        self.right_border = 0

    def move_left(self):
        """Moves left, updating the solution list accordingly."""
        if self.solution[-1] != 'L':
            self.solution.append('R')
        else:
            self.solution.pop() # this works only for n=2 and n=3
        self.pos = (self.pos - 1 + self.n) % self.n

    def move_right(self):
        """Moves right, updating the solution list accordingly."""
        if self.solution[-1] != 'R':
            self.solution.append('L')
        else:
            self.solution.pop() # this works only for n=2 and n=3
        self.pos = (self.pos + 1) % self.n

    def swap(self):
        """Swaps the current position."""
        self.solution.append('X')

    def solve(self, perm):
        """Executes the sorting process and returns the sequence of actions."""
        self.perm = perm
        self.n = len(perm)
        self.solution = ['X', 'L']
        self.pos = 1
        self.left_border = 2
        self.right_border = self.n - 1

        if self.n % 4 != 2 or self.n == 2:
            self._solve_case_one()
        else:
            self._solve_case_two()
        return self.solution

    def _solve_case_one(self):
        while self.right_border - self.left_border > self.n // 2:
            while self.pos != self.right_border:
                self.swap()
                self.move_left()
            self.right_border -= 1

            while self.pos != self.left_border:
                self.swap()
                self.move_right()
            self.left_border += 1

        while self.right_border - self.left_border > 0:
            while self.pos != self.right_border - 1:
                self.move_right()
                self.swap()
            self.right_border -= 1

            while self.pos != self.left_border:
                self.move_left()
                self.swap()
            self.left_border += 1

        while self.pos != 0:
            self.move_left()

    def _solve_case_two(self):
        while self.right_border - self.left_border > self.n // 2:
            while self.pos != self.right_border:
                self.swap()
                self.move_left()
            self.right_border -= 1

            while self.pos != self.left_border:
                self.swap()
                self.move_right()
            self.left_border += 1

        while self.pos != self.right_border:
            self.swap()
            self.move_left()
        self.left_border += 1

        while self.right_border - self.left_border > 0:
            while self.pos != self.left_border:
                self.move_left()
                self.swap()
            self.left_border += 1

            while self.pos != self.right_border - 1:
                self.move_right()
                self.swap()
            self.right_border -= 1

        while self.pos != 0:
            self.move_left()
        self.move_right()

In [12]:
test(200, BubbleSortCyclicSolver())

Total score:  1333298


In [13]:
visualize_solution_at_n(7, BubbleSortCyclicSolver())

Solution = XLXRXRXLXLXLLXLXRXRRR
n=7, len(moves)=21, diff with optimal=0.0
   |1|0 6 5 4 3 2
X: |0|1 6 5 4 3 2
L:  0|1|6 5 4 3 2
X:  0|6|1 5 4 3 2
R: |0|6 1 5 4 3 2
X: |6|0 1 5 4 3 2
R:  6 0 1 5 4 3|2|
X:  2 0 1 5 4 3|6|
L: |2|0 1 5 4 3 6
X: |0|2 1 5 4 3 6
L:  0|2|1 5 4 3 6
X:  0|1|2 5 4 3 6
L:  0 1|2|5 4 3 6
L:  0 1 2|5|4 3 6
X:  0 1 2|4|5 3 6
L:  0 1 2 4|5|3 6
X:  0 1 2 4|3|5 6
R:  0 1 2|4|3 5 6
X:  0 1 2|3|4 5 6
R:  0 1|2|3 4 5 6
R:  0|1|2 3 4 5 6
R: |0|1 2 3 4 5 6



In [14]:
visualize_solution_at_n(15, BubbleSortCyclicSolver(), tab_len=2)

Solution = XLXRXRXLXLXLXRXRXRXRXLXLXLXLXLXRXRXRXRXRXRXLXLXLXLXLXLXLLXLXLXLXLXLXRXRXRXRXRXLXLXLXLXRXRXRXLXLXRXRRRRRRR
n=15, len(moves)=105, diff with optimal=0.0
    |1| 0 14 13 12 11 10  9  8  7  6  5  4  3  2
X:  |0| 1 14 13 12 11 10  9  8  7  6  5  4  3  2
L:   0 |1|14 13 12 11 10  9  8  7  6  5  4  3  2
X:   0|14| 1 13 12 11 10  9  8  7  6  5  4  3  2
R:  |0|14  1 13 12 11 10  9  8  7  6  5  4  3  2
X: |14| 0  1 13 12 11 10  9  8  7  6  5  4  3  2
R:  14  0  1 13 12 11 10  9  8  7  6  5  4  3 |2|
X:   2  0  1 13 12 11 10  9  8  7  6  5  4  3|14|
L:  |2| 0  1 13 12 11 10  9  8  7  6  5  4  3 14
X:  |0| 2  1 13 12 11 10  9  8  7  6  5  4  3 14
L:   0 |2| 1 13 12 11 10  9  8  7  6  5  4  3 14
X:   0 |1| 2 13 12 11 10  9  8  7  6  5  4  3 14
L:   0  1 |2|13 12 11 10  9  8  7  6  5  4  3 14
X:   0  1|13| 2 12 11 10  9  8  7  6  5  4  3 14
R:   0 |1|13  2 12 11 10  9  8  7  6  5  4  3 14
X:   0|13| 1  2 12 11 10  9  8  7  6  5  4  3 14
R:  |0|13  1  2 12 11 10  9  8  7  6  5  4  3 14
X: |